In [4]:
#rtWIZARD Dry Run Protocol v1.0.0
#Will dry run protocol v1.0.0 using the same tip

#import opentrons.simulate if you want to simulate the protocol first
import opentrons.simulate
protocol = opentrons.simulate.get_protocol_api('2.13')
protocol.home()

#assigns each OT2 slot a custom labware definition that corresponds to the cutout
slot1 = protocol.load_labware("rtwizard_slots_147", location="1")
slot1.set_offset(x=1.30, y=-3.50, z=-4.00)

slot2 = protocol.load_labware("rtwizard_slots_258", location="2")
slot2.set_offset(x=-1.70, y=-2.30, z=-3.10)

slot3 = protocol.load_labware("rtwizard_slots_369", location="3")
slot3.set_offset(x=-5.00, y=-2.30, z=-3.70)

slot4 = protocol.load_labware("rtwizard_slots_147", location="4")
slot4.set_offset(x=1.00, y=-1.10, z=-4.00)

slot5 = protocol.load_labware("rtwizard_slots_258", location="5")
slot5.set_offset(x=-2.00, y=-1.00, z=-3.00)

slot6 = protocol.load_labware("rtwizard_slots_369", location="6")
slot6.set_offset(x=-5.00, y=-0.80, z=-3.00)

slot7 = protocol.load_labware("rtwizard_slots_147", location="7")
slot7.set_offset(x=-1.30, y=1.00, z=-4.00)

slot8 = protocol.load_labware("rtwizard_slots_258", location="8")
slot8.set_offset(x=-1.70, y=2.00, z=-3.60)

slot9 = protocol.load_labware("rtwizard_slots_369", location="9")
slot9.set_offset(x=-4.20, y=1.40, z=-3.00)

sample_plate = protocol.load_labware("rtwizard_48_wellplate_sarstedt_1100ul_tubes", location="10")
sample_plate.set_offset(x=0.00, y=1.00, z=-3.70)

tips = protocol.load_labware("opentrons_96_tiprack_20ul", location="11")
tips.set_offset(x=0.00, y=0.00, z=0.00)

#comment or uncomment depending on pipette setup
#left_pipette = protocol.load_instrument('p300_single_gen2', 'left', tip_racks=[tips])
#big_pipette = protocol.load_instrument('p1000_single_gen2', 'left', tip_racks=[tips])
small_pipette = protocol.load_instrument('p20_single_gen2', 'right', tip_racks=[tips])

def main():

    #user-defined values
    sample_vol = 20 #ul
    buffer_vol_per_drop = 20 #ul
    num_drops_buffer = 3
    
    #DRY RUN: pick up tip
    small_pipette.pick_up_tip(tips.wells()[0])
    
    #transfer sample liquid to cassettes
    transfer_samples(sample_vol)
    
    #transfer X drops of running buffer to cassettes
    for i in range(num_drops_buffer):
        transfer_buffer(buffer_vol_per_drop)
    
    #DRY RUN: drop tip
    small_pipette.drop_tip(tips.wells()[0])
    
    #rehome
    protocol.home()
    
    #print console output
    for line in protocol.commands():
        print(line)

#fin
        
def transfer_samples(sample_vol):
    for i in range(5):

        #determining which slots to dispense on for this iteration of the loop
        if (i == 0 or i == 1):
            start_slot = slot1
            mid_slot = slot4
            fin_slot = slot7
        elif (i == 2):
            start_slot = slot2
            mid_slot = slot5
            fin_slot = slot8
        else:
            start_slot = slot3
            mid_slot = slot6
            fin_slot = slot9

        #loading_wells: list of wells being loaded with sample liquid
        if ((i % 2 == 0 and i != 4) or i == 3):
            loading_wells = ['C1', 'B1', 'A1']
            loading_wells2 = ['A1', 'B1']
        else:
            loading_wells = ['C2', 'B2', 'A2']
            loading_wells2 = ['A2', 'B2']
        
        #transfers
        row = sample_plate.rows()[i]
        small_pipette.transfer(
            sample_vol, 
            row[:3], 
            [start_slot.wells_by_name()[well_name] for well_name in loading_wells],
            new_tip='always')
        small_pipette.transfer(
            sample_vol, 
            row[3:6], 
            [mid_slot.wells_by_name()[well_name] for well_name in loading_wells],
            new_tip='always')
        small_pipette.transfer(
            sample_vol, 
            row[6:8], 
            [fin_slot.wells_by_name()[well_name] for well_name in loading_wells2],
            new_tip='always')

    small_pipette.transfer(
        sample_vol, 
        [sample_plate.wells_by_name()[well_name] for well_name in ['F1', 'F2']], 
        [slot7.wells_by_name()[well_name] for well_name in ['A1', 'A2']],
        new_tip='always')

def transfer_buffer(buffer_vol_per_drop):
    for i in range(5):
        
        #determining which slots to dispense on for this iteration of the loop
        if (i == 0 or i == 1):
            start_slot = slot1
            mid_slot = slot4
            fin_slot = slot7
        elif (i == 2):
            start_slot = slot2
            mid_slot = slot5
            fin_slot = slot8
        else:
            start_slot = slot3
            mid_slot = slot6
            fin_slot = slot9

        #loading_wells: list of wells being loaded with running buffer
        if ((i % 2 == 0 and i != 4) or i == 3):
            loading_wells = ['C1', 'B1', 'A1']
            loading_wells2 = ['A1', 'B1']
        else:
            loading_wells = ['C2', 'B2', 'A2']
            loading_wells2 = ['A2', 'B2']

        #listing wells F3-F8 which contain running buffer
        buffer_wells = ['F3', 'F4', 'F5', 'F6', 'F7', 'F8']
        
        #distributes
        #TODO: determine if disposal_volume should be increased for more consistent dispense volumes at the cost of time and buffer
        small_pipette.distribute(
            buffer_vol_per_drop, 
            [sample_plate.wells_by_name()[well_name] for well_name in buffer_wells], 
            [start_slot.wells_by_name()[well_name] for well_name in loading_wells], 
            new_tip='never',
            disposal_volume=0)
        small_pipette.distribute(
            buffer_vol_per_drop, 
            [sample_plate.wells_by_name()[well_name] for well_name in buffer_wells], 
            [mid_slot.wells_by_name()[well_name] for well_name in loading_wells],
            new_tip='never',
            disposal_volume=0)
        small_pipette.distribute(
            buffer_vol_per_drop, 
            [sample_plate.wells_by_name()[well_name] for well_name in buffer_wells], 
            [fin_slot.wells_by_name()[well_name] for well_name in loading_wells2],
            new_tip='never',
            disposal_volume=0)

    small_pipette.distribute(
        buffer_vol_per_drop, 
        [sample_plate.wells_by_name()[well_name] for well_name in buffer_wells], 
        [slot7.wells_by_name()[well_name] for well_name in ['A1', 'A2']],
        new_tip='never',
        disposal_volume=0)
    
if __name__ == "__main__":
    main()

C:\Users\bhinn\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\bhinn\.opentrons\deck_calibration.json not found. Loading defaults


Picking up tip from A1 of Opentrons 96 Tip Rack 20 µL on 11
Transferring 20.0 from A1 of rtWIZARD 48 Wellplate with Sarstedt 1100ul Tubes on 10 to A1 of rtWIZARD Slots 1, 4, 7 on 1
Aspirating 20.0 uL from A1 of rtWIZARD 48 Wellplate with Sarstedt 1100ul Tubes on 10 at 7.56 uL/sec
Dispensing 20.0 uL into A1 of rtWIZARD Slots 1, 4, 7 on 1 at 7.56 uL/sec
Aspirating 20.0 uL from A2 of rtWIZARD 48 Wellplate with Sarstedt 1100ul Tubes on 10 at 7.56 uL/sec
Dispensing 20.0 uL into B1 of rtWIZARD Slots 1, 4, 7 on 1 at 7.56 uL/sec
Aspirating 20.0 uL from A3 of rtWIZARD 48 Wellplate with Sarstedt 1100ul Tubes on 10 at 7.56 uL/sec
Dispensing 20.0 uL into C1 of rtWIZARD Slots 1, 4, 7 on 1 at 7.56 uL/sec
Transferring 20.0 from A4 of rtWIZARD 48 Wellplate with Sarstedt 1100ul Tubes on 10 to A1 of rtWIZARD Slots 1, 4, 7 on 4
Aspirating 20.0 uL from A4 of rtWIZARD 48 Wellplate with Sarstedt 1100ul Tubes on 10 at 7.56 uL/sec
Dispensing 20.0 uL into A1 of rtWIZARD Slots 1, 4, 7 on 4 at 7.56 uL/sec
Aspira

In [ ]:
#run if you stop dry run protocol halfway through and need to return tip

#returns pipette tip and homes
import opentrons.execute
protocol = opentrons.execute.get_protocol_api('2.13')
protocol.home()

#assigns each OT2 slot a custom labware definition that corresponds to the cutout
slot1 = protocol.load_labware("rtwizard_slots_147", location="1")
slot1.set_offset(x=1.30, y=-3.50, z=-4.00)

slot2 = protocol.load_labware("rtwizard_slots_258", location="2")
slot2.set_offset(x=-1.70, y=-2.30, z=-3.10)

slot3 = protocol.load_labware("rtwizard_slots_369", location="3")
slot3.set_offset(x=-5.00, y=-2.30, z=-3.70)

slot4 = protocol.load_labware("rtwizard_slots_147", location="4")
slot4.set_offset(x=1.00, y=-1.10, z=-4.00)

slot5 = protocol.load_labware("rtwizard_slots_258", location="5")
slot5.set_offset(x=-2.00, y=-1.00, z=-3.00)

slot6 = protocol.load_labware("rtwizard_slots_369", location="6")
slot6.set_offset(x=-5.00, y=-0.80, z=-3.00)

slot7 = protocol.load_labware("rtwizard_slots_147", location="7")
slot7.set_offset(x=-1.30, y=1.00, z=-4.00)

slot8 = protocol.load_labware("rtwizard_slots_258", location="8")
slot8.set_offset(x=-1.70, y=2.00, z=-3.60)

slot9 = protocol.load_labware("rtwizard_slots_369", location="9")
slot9.set_offset(x=-4.20, y=1.40, z=-3.00)

sample_plate = protocol.load_labware("rtwizard_48_wellplate_sarstedt_1100ul_tubes", location="10")
sample_plate.set_offset(x=0.00, y=1.00, z=-3.70)

tips = protocol.load_labware("opentrons_96_tiprack_20ul", location="11")
tips.set_offset(x=0.00, y=0.00, z=0.00)

#comment or uncomment depending on pipette setup
#left_pipette = protocol.load_instrument('p300_single_gen2', 'left', tip_racks=[tips])
#big_pipette = protocol.load_instrument('p1000_single_gen2', 'left', tip_racks=[tips])
small_pipette = protocol.load_instrument('p20_single_gen2', 'right', tip_racks=[tips])

#DRY RUN: drop tip
small_pipette.drop_tip(tips.wells()[0])
    
#rehome
protocol.home()
    
#print console output
for line in protocol.commands():
    print(line)

